# Wildfire Data Scraping & Mapping for Geospatial Analysis

### Import Modules, authenticate, intialiaze and prep


In [ ]:
from datetime import datetime, timedelta
import ee
import geopandas as gpd
from IPython.display import display
import matplotlib.pyplot as plt
import os
import pandas as pd
from shapely.geometry import Point
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Authenticate & Initialize Earth Engine
#ee.Authenticate()
ee.Initialize(project='your-project-name') 

In [ ]:
# Display plots directly below cell
%matplotlib inline

In [ ]:
# Allow full column width display
pd.set_option('display.max_colwidth', None)

In [ ]:
# Check working directory
print(os.getcwd())

In [ ]:
# FIRMS API KEY
MAP_KEY = "your-FIRMS-mapkey-here"

In [ ]:
# Check number of available transactions
url_mapkey = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY

try:
  df_mapkey = pd.read_json(url_mapkey,  typ='series')
  display(df_mapkey)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url_mapkey)

In [ ]:
# Create function to check transaction usage

def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url_mapkey,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

### Data fetching and querying (Accessing FIRMS API and GEE)

In [ ]:
# Query data availability, instead of 'all' you can specify individual sensor, ex:MODIS_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df_sensors = pd.read_csv(da_url)
display(df_sensors)

In [ ]:
# Choose sensor(s) and store to variable(s), area and date range set at end of url or by methods in following code chunks
# Here we have set the dat range to the past 7 days, and left the area to world which we will subset later with more precise methods
url_modis_NRT = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{MAP_KEY}/MODIS_NRT/world/7/"

In [ ]:
# Set the url to a dataframe for analysis
df_modis = pd.read_csv(url_modis_NRT)
display(df_modis)

In [ ]:
# Convert the MODIS data into a GeoDataFrame (using 'longitude' and 'latitude' columns)
geometry = [Point(lon, lat) for lon, lat in zip(df_modis['longitude'], df_modis['latitude'])]
df_modis_gdf = gpd.GeoDataFrame(df_modis, geometry=geometry)

# Assign CRS to df_modis
df_modis_gdf = df_modis_gdf.set_crs("EPSG:4326", allow_override=True)

#### GEE

In [ ]:
# Retrieve countries Feature Collection from google earth engine to get proper clipping boundary
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
canada = countries.filter(ee.Filter.eq('country_na', 'Canada'))

# Fetch the geometry of Canada as a GeoJSON (alternative approach to the bounding box) and convert to GeoDataFrame
canada_geojson = canada.getInfo()
canada_gdf = gpd.GeoDataFrame.from_features(canada_geojson['features'])

# Assign the CRS to canada_gdf (WGS 84 - EPSG:4326)
canada_gdf = canada_gdf.set_crs("EPSG:4326", allow_override=True)

In [ ]:
# Clip the MODIS GeoDataFrame to the boundary of Canada using the Canada geometry
df_modis_clipped = gpd.clip(df_modis_gdf, canada_gdf)

In [ ]:
# Create total fire detections variable
total_fires = len(df_modis_clipped)

In [ ]:
# Assign date variables
today_date = datetime.now()
seven_days_ago = today_date - timedelta(days=6)

# Format the dates
formatted_date_today = today_date.strftime('%B, %d, %Y')
formatted_date_since = seven_days_ago.strftime('%B %d, %Y')

In [ ]:
# Define the extent for Canada
extent = [-150, 40, -49, 79]  

# Dissolve to remove internal lines
canada_outline = canada_gdf.dissolve()

# Create the base map and plot Canada boundary
ax = canada_outline.plot(figsize=(12, 10), color="lightgrey", edgecolor="blue", linewidth=1)

# Set map extent
ax.set_xlim([extent[0], extent[2]])
ax.set_ylim([extent[1], extent[3]])

# Plot clipped wildfire points
df_modis_clipped.plot(ax=ax, color="red", markersize=10, label="MODIS Wildfires")

# Add title, labels and legend
ax.set(title=f'A Total of {total_fires} Canada Wildfire Detections from {formatted_date_since} from {formatted_date_today}')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

plt.show()

### Custom Day/Night Detection Query

In [ ]:
# Subset data to values with confidence >= 50 and fire radiative power (frp) >= 5, for both day and night detections
df_custom_day = df_modis_clipped[(df_modis_clipped['confidence'] >= 50) & (df_modis_clipped['frp'] >= 5) & (df_modis_clipped['daynight'] == 'D')]
print ('Day time detection with confidence >=50 and frp >= 5 contains %i records' %  len(df_custom_day))

df_custom_night = df_modis_clipped[(df_modis_clipped['confidence'] >= 50) & (df_modis_clipped['frp'] >= 5) & (df_modis_clipped['daynight'] == 'N')]
print ('Night time detections with confidence >= 50 and frp >= 5 contains %i records' %  len(df_custom_night))


In [ ]:
# Create one figure and axis for all plots
fig, ax = plt.subplots(figsize=(12, 10))

# Create the basemap and plot Canada boundary
canada_outline.plot(ax=ax, color="lightgrey", edgecolor='blue', linewidth=1, label='Canada Boundary')

# Set map extent
ax.set_xlim([extent[0], extent[2]])
ax.set_ylim([extent[1], extent[3]])

# Plot detections
df_custom_day.plot(ax=ax, color='red', markersize=10, label='Daytime Detections')
df_custom_night.plot(ax=ax, color='blue', markersize=10, label='Nighttime Detections')

# Add title, labels, legend
ax.set_title(f'MODIS Detections by Day/Night \nCanada Wildfires Detections from {formatted_date_since} to {formatted_date_today}')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

plt.show()

### Adding datetime column in dataset

In [ ]:
# Create datetime column in dataset combining acq_date and acq_time
df_modis_clipped['acq_datetime'] = pd.to_datetime(df_modis_clipped['acq_date'] + ' ' + df_modis_clipped['acq_time'].astype(str).str.zfill(4), format='%Y-%m-%d %H%M')

print ('Canada sample datetime info:')
df_modis_clipped['acq_datetime'].sample(5)

In [ ]:
df_modis_clipped.head()

In [ ]:
# Display min/max date time range for df_modis_clipped
print ('Canada datetime value range: %s to %s' % (str(df_modis_clipped['acq_datetime'].min()), str(df_modis_clipped['acq_datetime'].max())))

### Analyze Time Since Detection

In [ ]:
# Import timezone library for Canada
import pytz

print('Canada TimeZones')
for timeZone in pytz.country_timezones['CA']:
    print(timeZone)

In [ ]:
# Compare min/max date-time range converted using 3 different Canada time zones
# Original GMT
# Halifax (GMT-3:00)
# Toronto (GMT-4:00)
# Vancouver (GMT-7:00)

print('Canada GMT timezone datetime value range: %s to %s' % (str(df_modis_clipped['acq_datetime'].min()), str(df_modis_clipped['acq_datetime'].max())))
print('Canada Halifax timezone datetime value range: %s to %s' % (str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Halifax').min()), str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Halifax').max())))
print('Canada Toronto timezone datetime value range: %s to %s' % (str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Toronto').min()), str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Toronto').max())))
print('Canada Vancouver timezone datetime value range: %s to %s' % (str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Vancouver').min()), str(df_modis_clipped['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Vancouver').max())))


In [ ]:
# Assign detection time max to variable
dt_max = df_modis_clipped['acq_datetime'].max()

In [ ]:
# Create subsets for colour classes

# <= 1 hour
df_modis_clipped1 = df_modis_clipped[df_modis_clipped['acq_datetime'] >= (dt_max - pd.Timedelta(hours=1))]

# > 1 hour and <= 4 hours
df_modis_clipped2 = df_modis_clipped[(df_modis_clipped['acq_datetime'] >= (dt_max - pd.Timedelta(hours=4))) & (df_modis_clipped['acq_datetime'] < (dt_max - pd.Timedelta(hours=1)))]

# > 4 hours and <= 12 hours
df_modis_clipped3 = df_modis_clipped[(df_modis_clipped['acq_datetime'] >= (dt_max - pd.Timedelta(hours=12))) & (df_modis_clipped['acq_datetime'] < (dt_max - pd.Timedelta(hours=4)))]

# > 12 hours
df_modis_clipped4 = df_modis_clipped[df_modis_clipped['acq_datetime'] < (dt_max - pd.Timedelta(hours=12))]


print ('Counts per subset: %i, %i, %i, %i from total of %i' % (df_modis_clipped1.count()[0],df_modis_clipped2.count()[0],df_modis_clipped3.count()[0],df_modis_clipped4.count()[0], df_modis_clipped.count()[0]))

In [ ]:
# Create the basemap and plot Canada Boundary
ax = canada_outline.plot(figsize=(12, 10), color="lightgrey", edgecolor="blue", linewidth=1)

# Set map extent
ax.set_xlim([extent[0], extent[2]])
ax.set_ylim([extent[1], extent[3]])

# Color code each set; also we are drawing in opposite order, so the older detections are drawn first so the newer ones are on top
df_modis_clipped4.plot(ax=ax, color="yellow", markersize=10, label='More Than 12 Hours Ago')
df_modis_clipped3.plot(ax=ax, color="orange", markersize=10, label='4-12 Hours Ago')
df_modis_clipped2.plot(ax=ax, color="red", markersize=10, label='1-4 Hours Ago')
df_modis_clipped1.plot(ax=ax, color="darkred", markersize=10, label='1 Hour Ago')

# Add title, labels, legend
ax.set(title=f'Time Since Detection\nCanada Wildfires from {formatted_date_since} to {formatted_date_today}')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend(title="Time Since Detection", loc='upper right')

plt.show()